In [65]:
from basic_rl_envi_for_dam_bidding_main.import_data_example import *

# Get Data

In [87]:
start = pd.Timestamp(year=2023, month=3, day=1, tz="utc")
#end = pd.Timestamp(year=2022, month=5, day=1, tz="utc")
end = pd.Timestamp.now(tz="utc")
# get the end of the current day
end = end.floor('D') + pd.Timedelta(days=3)

In [88]:
df_demand = get_demand(start, end)
# wann wird der forecast available? Basierend darauf muss später gejoined werden
df_mcp = get_mcp(start, end)
df_vre = get_vre(start, end)
df_gen = get_gen(start, end)
#states = get_states_list(start, end)

#checken ob daten um 24:00 Uhr kommen oder um 12:00 Uhr

In [89]:
# füge eine variable hinzu, die angibt zu welchem Zeitpunkt die Daten verfügbar sind
df_vre

,Forecasted Load
2018-10-01 02:00:00+02:00,42696.0
2018-10-01 02:15:00+02:00,42437.0
2018-10-01 02:30:00+02:00,42691.0
2018-10-01 02:45:00+02:00,42688.0
2018-10-01 03:00:00+02:00,42747.0
...,...
2023-05-09 22:45:00+02:00,50338.0
2023-05-09 23:00:00+02:00,49402.0
2023-05-09 23:15:00+02:00,48619.0
2023-05-09 23:30:00+02:00,47939.0


In [79]:
df_vre['day available']

,Forecasted Load
2022-03-01 00:00:00+01:00,53995.0
2022-03-01 00:15:00+01:00,53333.0
2022-03-01 00:30:00+01:00,52738.0
2022-03-01 00:45:00+01:00,52367.0
2022-03-01 01:00:00+01:00,51878.0
...,...
2023-05-09 22:45:00+02:00,50338.0
2023-05-09 23:00:00+02:00,49402.0
2023-05-09 23:15:00+02:00,48619.0
2023-05-09 23:30:00+02:00,47939.0


In [39]:
df_demand_hourly

In [90]:
# get day ahead market price for each day
df_mcp_hourly = df_mcp.pivot_table(index=df_mcp.index.date, columns=df_mcp.index.hour, values=0)
df_mcp_hourly.index = pd.to_datetime(df_mcp_hourly.index)
df_mcp_hourly = df_mcp_hourly.add_prefix('market_price_hour_')
df_mcp_hourly['end_auction'] = (df_mcp_hourly.index - pd.Timedelta(hours = 12))
#df_mcp_hourly = df_mcp_hourly.reset_index(drop=False).set_index('end_auction')
#df_mcp_hourly = df_mcp_hourly.rename(columns={'index': 'day'})

In [91]:
# df_demand: 2023-05-08 23:45:00+02:00 gibt den Load für 23:45-00:00 Uhr an
# 2023-05-08 12:00:00+02:00,63621.0,29065.0,2153.0,10199.0,63511.0,80.41
# dieser Datenpunkt entspricht den Werten von 12:00 Uhr bis 12:30 Uhr

In [42]:
# get forecasted load for each hour of each day.
df_demand_hourly = df_demand.pivot_table(index=df_demand.index.date, columns=df_demand.index.hour, values='Forecasted Load', aggfunc='sum')
df_demand_hourly = df_demand_hourly.add_prefix('forecasted_load_hour_')

In [43]:
# aggregate hourly
df_vre_agg = df_vre.groupby([pd.Grouper(freq='1h')]).sum()

In [102]:
df_vre_hourly = df_vre_agg.pivot_table(index=df_vre_agg.index.date, columns=df_vre_agg.index.hour, values=['Wind Onshore', 'Wind Offshore', 'Solar'], aggfunc='sum')
# add prefix and concatenate df
df_vre_hourly = pd.concat([df_vre_hourly[col].add_prefix(f'{col}_hour_') for col in df_vre_hourly.columns.get_level_values(level=0).unique()], axis=1)

In [103]:
df_gen_hourly = df_gen.pivot_table(index=df_gen.index.date, columns=df_gen.index.hour, values='Actual Aggregated', aggfunc='sum')
df_gen_hourly = df_gen_hourly.add_prefix('actual_aggregated_hour_')

In [104]:
#
l = [df_demand_hourly, df_vre_hourly, df_gen_hourly, df_mcp_hourly]

In [105]:
# ab wann sind die Daten verfügbar?

# um 17:15 am 8.Mai
# df_demand_hourly bis 2023-05-09 23:00
# df_vre_hourly bis 2023-05-08 23:00
# df_gen_hourly bis 2023-05-08 23:00
# df_mcp_hourly bis 2023-05-09 23:00


,forecasted_load_hour_0,forecasted_load_hour_1,forecasted_load_hour_2,forecasted_load_hour_3,forecasted_load_hour_4,forecasted_load_hour_5,forecasted_load_hour_6,forecasted_load_hour_7,forecasted_load_hour_8,forecasted_load_hour_9,...,market_price_hour_14,market_price_hour_15,market_price_hour_16,market_price_hour_17,market_price_hour_18,market_price_hour_19,market_price_hour_20,market_price_hour_21,market_price_hour_22,market_price_hour_23
end_auction,,,,,,,,,,,,,,,,,,,,,
2018-10-11 12:00:00,201063.0,192996.0,189549.0,190698.0,196010.0,208553.0,238030.0,259510.0,269073.0,272494.0,...,47.84,53.32,56.95,67.32,69.94,79.08,65.00,53.58,50.00,43.88
2018-10-12 12:00:00,190250.0,181665.0,176778.0,175878.0,176937.0,178868.0,184744.0,197375.0,212858.0,223986.0,...,31.95,37.42,50.23,60.46,64.54,55.85,40.59,31.52,24.72,20.29
2018-10-13 12:00:00,175236.0,167669.0,163901.0,162658.0,164176.0,164842.0,167321.0,176114.0,189478.0,201767.0,...,13.92,19.54,20.74,42.82,56.56,68.09,40.54,32.90,31.32,27.00
2018-10-14 12:00:00,177229.0,171728.0,169473.0,170771.0,178232.0,195459.0,231218.0,255873.0,266980.0,269708.0,...,51.45,55.26,71.86,80.28,90.00,107.32,75.31,70.49,59.05,59.07
2018-10-15 12:00:00,193888.0,187322.0,183825.0,184485.0,190204.0,203066.0,234806.0,258282.0,268794.0,272032.0,...,68.61,69.19,74.04,80.95,89.97,105.12,84.96,76.02,59.99,56.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-04 12:00:00,187268.0,180504.0,177404.0,178729.0,183737.0,195244.0,218627.0,239211.0,250013.0,252828.0,...,78.90,81.08,91.16,100.17,112.80,120.54,128.57,126.31,115.56,103.34
2023-05-05 12:00:00,178686.0,170291.0,165397.0,165289.0,165567.0,166777.0,170943.0,180339.0,194227.0,205778.0,...,50.00,63.16,74.64,90.30,102.70,108.05,111.81,108.71,103.12,92.38
2023-05-06 12:00:00,156546.0,150069.0,146385.0,145810.0,146519.0,148029.0,149051.0,156625.0,169065.0,181813.0,...,-0.03,15.44,56.62,75.97,94.94,99.36,106.45,105.02,99.30,96.07


In [108]:
df_mcp_hourly

In [110]:
# join der Daten ist nicht korrekt
df = pd.concat([df_demand_hourly, df_vre_hourly, df_gen_hourly, df_mcp_hourly], axis=1)

In [266]:
df

,forecasted_load_hour_0,forecasted_load_hour_1,forecasted_load_hour_2,forecasted_load_hour_3,forecasted_load_hour_4,forecasted_load_hour_5,forecasted_load_hour_6,forecasted_load_hour_7,forecasted_load_hour_8,forecasted_load_hour_9,...,market_price_hour_15,market_price_hour_16,market_price_hour_17,market_price_hour_18,market_price_hour_19,market_price_hour_20,market_price_hour_21,market_price_hour_22,market_price_hour_23,end_auction
2023-03-01,NaN,202201.0,199722.0,201209.0,205615.0,217546.0,242339.0,259557.0,269253.0,273495.0,...,122.81,135.46,162.74,183.69,182.48,162.74,144.85,142.05,131.51,2023-02-28 12:00:00
2023-03-02,210615.0,203958.0,201056.0,202722.0,207651.0,220122.0,245990.0,263352.0,273116.0,275667.0,...,128.62,152.64,166.38,184.91,184.13,163.43,159.31,149.53,139.90,2023-03-01 12:00:00
2023-03-03,209231.0,202353.0,200215.0,201846.0,207412.0,219564.0,242874.0,261214.0,271378.0,273974.0,...,114.69,129.31,155.98,165.00,152.94,133.31,122.20,112.09,101.63,2023-03-02 12:00:00
2023-03-04,208905.0,201157.0,197389.0,196285.0,196250.0,196526.0,201076.0,209615.0,221057.0,231850.0,...,101.77,106.90,140.00,151.66,145.00,138.68,117.71,121.02,109.41,2023-03-03 12:00:00
2023-03-05,184430.0,178168.0,174895.0,174166.0,174403.0,175287.0,175933.0,182501.0,194247.0,206916.0,...,115.14,129.37,145.14,166.67,172.37,160.23,152.49,143.66,130.13,2023-03-04 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81.08,91.16,100.17,112.80,120.54,128.57,126.31,115.56,103.34,2023-05-04 12:00:00
2023-05-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,63.16,74.64,90.30,102.70,108.05,111.81,108.71,103.12,92.38,2023-05-05 12:00:00
2023-05-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.44,56.62,75.97,94.94,99.36,106.45,105.02,99.30,96.07,2023-05-06 12:00:00
2023-05-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,78.00,84.01,97.05,110.04,132.90,128.41,121.99,112.56,99.38,2023-05-07 12:00:00


In [264]:
df

,forecasted_load_hour_0,forecasted_load_hour_1,forecasted_load_hour_2,forecasted_load_hour_3,forecasted_load_hour_4,forecasted_load_hour_5,forecasted_load_hour_6,forecasted_load_hour_7,forecasted_load_hour_8,forecasted_load_hour_9,...,actual_aggregated_hour_14,actual_aggregated_hour_15,actual_aggregated_hour_16,actual_aggregated_hour_17,actual_aggregated_hour_18,actual_aggregated_hour_19,actual_aggregated_hour_20,actual_aggregated_hour_21,actual_aggregated_hour_22,actual_aggregated_hour_23
2023-03-01,NaN,202201.0,199722.0,201209.0,205615.0,217546.0,242339.0,259557.0,269253.0,273495.0,...,65345.0,62459.0,60284.0,59173.0,59953.0,61257.0,60815.0,59067.0,55816.0,52488.0
2023-03-02,210615.0,203958.0,201056.0,202722.0,207651.0,220122.0,245990.0,263352.0,273116.0,275667.0,...,64766.0,62527.0,59583.0,57823.0,60239.0,60564.0,58558.0,56498.0,53556.0,50519.0
2023-03-03,209231.0,202353.0,200215.0,201846.0,207412.0,219564.0,242874.0,261214.0,271378.0,273974.0,...,63956.0,63140.0,60778.0,61207.0,64438.0,65723.0,63449.0,60150.0,56615.0,55302.0
2023-03-04,208905.0,201157.0,197389.0,196285.0,196250.0,196526.0,201076.0,209615.0,221057.0,231850.0,...,63282.0,59050.0,56785.0,57083.0,58042.0,56685.0,53738.0,51005.0,49924.0,47419.0
2023-03-05,184430.0,178168.0,174895.0,174166.0,174403.0,175287.0,175933.0,182501.0,194247.0,206916.0,...,53158.0,51560.0,51022.0,50149.0,51444.0,51724.0,51493.0,49932.0,49412.0,46521.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Bedeutung der einzelnen Variablen
- Forecasted Load:
- Solar
- Wind Offshore:
- Wind Onshore:
- day ahead market price: EUR/MWh
